In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

#Stack Data

In [ ]:
class Stock:
    def __init__(self, stack_name, start="2021-01-01", end="2024-01-14"):
        self.stack_name = stack_name
        self.start = start
        self.end = end
        self.data = yf.download(self.stack_name, self.start, self.end).reset_index()
        self.maximum=self.data.High.max()
        self.minimum=self.data.Low.min()
        self.lr = LinearRegression()

    def stock_chart(self):
        plt.plot(self.data['Close'])
        plt.title(f"{self.stack_name} Stock chart")
        plt.xlabel("Date")
        plt.ylabel("Price (INR)")
        plt.show()

    def stock_performance(self):
        x = np.array(self.data.index).reshape(-1, 1)
        y = np.array(self.data['Close'])
        print("Linear score is:", self.lr.fit(x, y).score(x, y))
        plt.scatter(x, y)
        plt.plot(x, self.lr.predict(x))
        plt.show()

In [ ]:
stock_instance = Stock(stack_name="AAPL")
stock_instance.stock_chart()
stock_instance.stock_performance()

In [ ]:
nifty = Stock("^NSEI ")  # nifty50
print("Nifty 50 maximum value is: ",nifty.maximum)
print(nifty.minimum)
nifty.stock_chart()
nifty.stock_performance()

In [ ]:
itc= Stock("ITC.NS",start="2023-01-01")
print("ITC Minimum value is:",itc.minimum)
print(itc.maximum)

[*********************100%%**********************]  1 of 1 completed
ITC Minimum value is: 326.0
499.70001220703125


#LSTM

In [ ]:
# Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
import yfinance as yf

In [ ]:
# Load INFY stock data
infy_data = yf.download('INFY', start="2021-01-01", end="2024-01-14")
df = infy_data['Close'].values.reshape(-1, 1)

[*********************100%%**********************]  1 of 1 completed


In [ ]:
# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
df = scaler.fit_transform(df)

In [ ]:
# Create function to create dataset for LSTM
def create_dataset(dataset, time_step=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step), 0]
        dataX.append(a)
        dataY.append(dataset[i + time_step, 0])
    return np.array(dataX), np.array(dataY)

In [ ]:

# Reshape data for LSTM
time_step = 100
X, y = create_dataset(df, time_step)


In [ ]:

# Reshape input to be [samples, time steps, features] which is required for LSTM
X = X.reshape(X.shape[0], X.shape[1], 1)


In [ ]:

# Build the LSTM model
model = Sequential()
model.add(LSTM(100, input_shape=(X.shape[1], 1), return_sequences=True))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(50))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')


In [ ]:

# Train the model
model.fit(X, y, epochs=10, batch_size=64, verbose=1)


Epoch 1/10
11/11 [==============================] - 7s 203ms/step - loss: 0.0594
Epoch 2/10
11/11 [==============================] - 2s 205ms/step - loss: 0.0148
Epoch 3/10
11/11 [==============================] - 3s 239ms/step - loss: 0.0080
Epoch 4/10
11/11 [==============================] - 2s 204ms/step - loss: 0.0064
Epoch 5/10
11/11 [==============================] - 2s 200ms/step - loss: 0.0058
Epoch 6/10
11/11 [==============================] - 2s 198ms/step - loss: 0.0058
Epoch 7/10
11/11 [==============================] - 2s 229ms/step - loss: 0.0053
Epoch 8/10
11/11 [==============================] - 3s 214ms/step - loss: 0.0048
Epoch 9/10
11/11 [==============================] - 2s 201ms/step - loss: 0.0048
Epoch 10/10
11/11 [==============================] - 2s 201ms/step - loss: 0.0043


In [ ]:
# Predict the next 10 days
future_days = 10
x_input = df[-time_step:]
x_input.shape

(100, 1)

In [ ]:
x_input = scaler.transform(x_input)
x_input = x_input.reshape(1, time_step, 1)

In [ ]:
predicted_values = []
for i in range(future_days):
    yhat = model.predict(x_input, verbose=0)
    predicted_values.append(yhat[0, 0])
    x_input = np.append(x_input[:, 1:, :], [[yhat[0, 0]]], axis=1)

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 3 dimension(s) and the array at index 1 has 2 dimension(s)

In [ ]:
# Inverse transform the predicted values
predicted_values = scaler.inverse_transform(np.array(predicted_values).reshape(-1, 1))

In [ ]:
# Plotting
day_new = np.arange(len(df), len(df) + future_days)
day_pred = np.arange(len(df), len(df) + future_days)

In [ ]:
plt.plot(df, label='Actual Data')
plt.plot(day_pred, predicted_values, label='Predicted Data')
plt.legend()
plt.show()


#Get data from stock related news use NLP and predict